<a href="https://colab.research.google.com/github/ishanyaa/Grovers_Algorithm/blob/main/Grover's_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
!pip install pennylane --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.8 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

Rough Work: https://colab.research.google.com/drive/1_CCvw06YmQ106I6jNX0wy9gi4aeQLYUL?usp=sharing

# Ishanya

In [ ]:
# Ishanya- 2 Qubit Grover's Search Algorithm

n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits, shots=1024)

""" Oracle function """

def oracle(marked_state):
    # Convert the marked state to binary
    marked_binary = f"{marked_state:0{n_qubits}b}"
    # Apply X gates to flip the marked state
    for i, bit in enumerate(marked_binary):
        if bit == "0":
            qml.PauliX(wires=i)
    # Apply the multi-controlled Z gate
    qml.ctrl(qml.PauliZ, control=[0])(1)
    # Apply X gates to reset controls
    for i, bit in enumerate(marked_binary):
        if bit == "0":
            qml.PauliX(wires=i)

""" Grover diffusion operator"""
def grover_diffusion():
    # Apply Hadamard to all qubits
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    # Apply Z gate to flip the sign of all states
    for i in range(n_qubits):
        qml.PauliZ(wires=i)

    # Apply a controlled-Z gate to create the reflection
    qml.ctrl(qml.PauliZ, control=[0])(1)
    # Apply Hadamard again
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

""" Quantum circuit"""
@qml.qnode(dev)
def grover_circuit(marked_state):
    # Step 1: Initialize superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Step 2: Apply the oracle
    oracle(marked_state)

    # Step 3: Apply the Grover diffusion operator
    grover_diffusion()

    # Measure the state
    return qml.probs(wires=range(n_qubits))


# User inpt
try:
    marked_state = int(input(f"Enter the marked state (0 to {2**n_qubits - 1}): "))
    if marked_state < 0 or marked_state >= 2**n_qubits:
        raise ValueError("Marked state out of range.")
except ValueError as e:
    print("Invalid input. Defaulting to state |11>.")
    marked_state = 3

probs = grover_circuit(marked_state)

# Plot
states = [f"{bin(i)[2:].zfill(n_qubits)}" for i in range(2**n_qubits)]
plt.bar(states, probs, color="purple")
plt.xlabel("States")
plt.ylabel("Probability")
plt.title(f"Grover's Algorithm Output (Marked State: |{bin(marked_state)[2:].zfill(n_qubits)}⟩)")
plt.show()

# Circuit visualization
print("Quantum Circuit:")
drawer = qml.draw(grover_circuit)
print(drawer(marked_state))

In [ ]:
# Ishanya- 3 SAT (Code works!)

import time
from typing import List, Tuple

#USING GROVER'S ALGORITHM!!

def _not(y):
    return y ^ 1

def sat_formula(x1, x2, x3):
    return (
        ((x1) | (x2) | (x3))
        & ((_not(x1)) | (x2) | (x3))
        & ((_not(x1)) | (_not(x2)) | (_not(x3)))
        & ((_not(x1)) | (_not(x2)) | (x3))
        & ((x1) | (x2) | (_not(x3)))
        & ((_not(x1)) | (x2) | (_not(x3)))
    )

NUM_VARIABLES = 3

def create_oracle_matrix():
    """
    Create oracle matrix for the specific SAT formula
    The oracle matrix-row corresponds to a possible assignment of the variables (x1, x2, x3), and each column corresponds to a state in the superposition of all possible assignments.
 oracle matrix creation is checking if each assignment satisfies the SAT formula. If the assignment satisfies the formula, the phase of that state is flipped to -1; otherwise, it stays +1.
    """
    N = 2**NUM_VARIABLES
    oracle_matrix = np.zeros((N, N), dtype=complex)

    for i in range(N):
        # Convert number to binary representation and get individual bits
        bits = format(i, f'0{NUM_VARIABLES}b')
        x1, x2, x3 = [int(b) for b in bits]

        # Check if this assignment satisfies the formula
        satisfies = sat_formula(x1, x2, x3)

        # Apply phase flip if solution is found
        oracle_matrix[i, i] = -1 if satisfies else 1

    return oracle_matrix

def diffusion_operator():
    """
    Create diffusion operator matrix for Grover's algorithm
    Creating a matrix of size 2^3 * 2^3 =8×8, where each element is initialized to 2/𝑁.
    The diagonal is set to (−1+𝑁/2) for amplitude inversion.
    """
    N = 2**NUM_VARIABLES
    diffusion = np.full((N, N), 2/N)
    np.fill_diagonal(diffusion, -1 + 2/N)
    return diffusion

def grover_sat_solver(steps: int = None, shots: int = 1000) -> Tuple[str, float, List[str]]:
    """
    Solve SAT problem using Grover's algorithm
    Returns:
        Tuple of (solution_string, probability, all_solutions)
    """
    # optimal number of steps
    if steps is None:
        N = 2**NUM_VARIABLES
        # For this specific formula, we can count solutions
        solution_count = 0
        for i in range(N):
            bits = format(i, f'0{NUM_VARIABLES}b')
            x1, x2, x3 = [int(b) for b in bits]
            if sat_formula(x1, x2, x3):
                solution_count += 1
        steps = int(np.pi/4 * np.sqrt(N/max(1, solution_count))) #this is for opt iterations.

    dev = qml.device('default.qubit', wires=NUM_VARIABLES, shots=shots)

    oracle = create_oracle_matrix()
    diffusion = diffusion_operator()

    @qml.qnode(dev)
    def circuit():
        # Initialize superposition
        for i in range(NUM_VARIABLES):
            qml.Hadamard(wires=i)

        #  iterations
        for _ in range(steps):
            # Oracle
            qml.QubitUnitary(oracle, wires=range(NUM_VARIABLES))

            # Diffusion
            qml.QubitUnitary(diffusion, wires=range(NUM_VARIABLES))

        return qml.probs(wires=range(NUM_VARIABLES))

    # Run circuit and get probabilities
    probs = circuit()

    # Get most likely solution
    solution_idx = np.argmax(probs)
    solution = format(solution_idx, f'0{NUM_VARIABLES}b')

    # Find all valid solutions for verification (why is not finding all assignments ah)
    all_solutions = []
    for i in range(2**NUM_VARIABLES):
        bits = format(i, f'0{NUM_VARIABLES}b')
        x1, x2, x3 = [int(b) for b in bits]
        if sat_formula(x1, x2, x3):
            all_solutions.append(bits)

    return solution, probs[solution_idx], all_solutions

def verify_solution(solution: str) -> bool:
    """
    Verify if a solution satisfies the SAT formula
    """
    x1, x2, x3 = [int(b) for b in solution]
    return sat_formula(x1, x2, x3)

if __name__ == "__main__":
    print("Starting 3-SAT solver using Grover's algorithm...")
    print(f"Number of variables: {NUM_VARIABLES}")

    start_time = time.time()

""" Oracle creation time"""
    oracle_start = time.time()
    oracle = create_oracle_matrix()
    oracle_time = time.time() - oracle_start
    print(f"\nOracle creation time: {oracle_time:.4f} seconds")

    solve_start = time.time()
    solution, probability, all_solutions = grover_sat_solver()
    solve_time = time.time() - solve_start

    total_time = time.time() - start_time

    print(f"\nResults:")
    print(f"Found solution: {solution}")
    print(f"Probability: {probability:.4f}")
    print(f"Solution is valid: {verify_solution(solution)}")

    print("\nVariable assignments:")
    variables = ['x1', 'x2', 'x3']
    for i, bit in enumerate(solution):
        print(f"{variables[i]} = {bit}")

    print(f"\nAll valid solutions found: {all_solutions}")
    print(f"Number of valid solutions: {len(all_solutions)}")

    print(f"\nTiming information:")
    print(f"Oracle creation time: {oracle_time:.4f} seconds")
    print(f"Solution time: {solve_time:.4f} seconds")
    print(f"Total execution time: {total_time:.4f} seconds")

    if solution in all_solutions:
        print("\nVerification: Solution found is correct and matches classical verification")
    else:
        print("\nWarning: Solution found does not match classical verification")

In [ ]:
#Ishanya

NUM_QUBITS = 10
N = 2**NUM_QUBITS
wires = list(range(NUM_QUBITS))

# user input
print(f"Enter the marked states as binary strings of length {NUM_QUBITS}.")
print("Enter one state per line and type 'done' when finished:")

marked_states = []
while True:
    user_input = input("Marked state: ").strip()
    if user_input.lower() == "done":
        break
    if len(user_input) != NUM_QUBITS or not set(user_input).issubset({"0", "1"}):
        print(f"Invalid input. Please enter a binary string of length {NUM_QUBITS}.")
        continue
    marked_states.append([int(bit) for bit in user_input])

if not marked_states:
    print("No marked states provided. Exiting.")
    exit()

omega = np.array(marked_states)
M = len(omega)

dev = qml.device("default.qubit", wires=NUM_QUBITS)

def oracle(wires, omega):
    qml.FlipSign(omega, wires=wires)

def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

@qml.qnode(dev)
def circuit():
    iterations = int(np.round(np.sqrt(N / M) * np.pi / 4))

    equal_superposition(wires)

    for _ in range(iterations):
        for omg in omega:
            oracle(wires, omg)
        qml.templates.GroverOperator(wires)

    return qml.probs(wires)

probs = circuit()

''' Create a list of tuples (probability, binary_string)'''
state_probs = []
for index, prob in enumerate(probs):
    binary_string = format(index, f"0{NUM_QUBITS}b")
    if prob > 1e-6:
        state_probs.append((prob, binary_string))

# Sort in descending order
state_probs.sort(reverse=True, key=lambda x: x[0])

print("\nMeasured probabilities for each state (sorted in descending order):")
for prob, binary_string in state_probs:
    print(f"State {binary_string}: Probability {prob:.6f}")

Enter the marked states as binary strings of length 10.
Enter one state per line and type 'done' when finished:
Invalid input. Please enter a binary string of length 10.


In [ ]:
#Ishanya

NUM_QUBITS = 10
N = 2**NUM_QUBITS
wires = list(range(NUM_QUBITS))

# user input
print(f"Enter the marked states as binary strings of length {NUM_QUBITS}.")
print("Enter one state per line and type 'done' when finished:")

marked_states = []
while True:
    user_input = input("Marked state: ").strip()
    if user_input.lower() == "done":
        break
    if len(user_input) != NUM_QUBITS or not set(user_input).issubset({"0", "1"}):
        print(f"Invalid input. Please enter a binary string of length {NUM_QUBITS}.")
        continue
    marked_states.append([int(bit) for bit in user_input])

if not marked_states:
    print("No marked states provided. Exiting.")
    exit()

omega = np.array(marked_states)
M = len(omega)

dev = qml.device("default.qubit", wires=NUM_QUBITS)

def oracle(wires, omega):
    qml.FlipSign(omega, wires=wires)

def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

@qml.qnode(dev)
def circuit():
    iterations = int(np.round(np.sqrt(N / M) * np.pi / 4))

    equal_superposition(wires)

    for _ in range(iterations):
        for omg in omega:
            oracle(wires, omg)
        qml.templates.GroverOperator(wires)

    return qml.probs(wires)

probs = circuit()

'''Create a list of tuples (probability, binary_string)'''
state_probs = []
for index, prob in enumerate(probs):
    binary_string = format(index, f"0{NUM_QUBITS}b")
    if prob > 1e-6:  # Consider only probabilities greater than a small threshold
        state_probs.append((prob, binary_string))

# Sort in descending order
state_probs.sort(reverse=True, key=lambda x: x[0])

print("\nMeasured probabilities for each state (sorted in descending order):")
for prob, binary_string in state_probs:
    print(f"State {binary_string}: Probability {prob:.6f}")

Enter the marked states as binary strings of length 10.
Enter one state per line and type 'done' when finished:
Marked state: 110101010
Invalid input. Please enter a binary string of length 10.
Marked state: 1111111111
Marked state: 0000000000
Marked state: done

Measured probabilities for each state (sorted in descending order):
State 0000000000: Probability 0.497896
State 1111111111: Probability 0.497896
State 0000000001: Probability 0.000004
State 0000000010: Probability 0.000004
State 0000000011: Probability 0.000004
State 0000000100: Probability 0.000004
State 0000000101: Probability 0.000004
State 0000000110: Probability 0.000004
State 0000000111: Probability 0.000004
State 0000001000: Probability 0.000004
State 0000001001: Probability 0.000004
State 0000001010: Probability 0.000004
State 0000001011: Probability 0.000004
State 0000001100: Probability 0.000004
State 0000001101: Probability 0.000004
State 0000001110: Probability 0.000004
State 0000001111: Probability 0.000004
State

In [ ]:
# Ishanya

NUM_QUBITS = 10
N = 2**NUM_QUBITS
wires = list(range(NUM_QUBITS))

# Take user input - in decimal.
print(f"Enter the marked states as decimal integers, each representing a binary string of length {NUM_QUBITS}.")
print("Enter one state per line and type 'done' when finished:")

marked_states = []
while True:
    user_input = input("Marked state (decimal): ").strip()
    if user_input.lower() == "done":
        break
    try:
        decimal_input = int(user_input)  # Convert input to an integer
        if decimal_input >= 2**NUM_QUBITS or decimal_input < 0:
            print(f"Invalid input. Please enter a decimal integer between 0 and {2**NUM_QUBITS - 1}.")
            continue
        # Convert decimal to binary
        binary_string = format(decimal_input, f"0{NUM_QUBITS}b")
        marked_states.append([int(bit) for bit in binary_string])
    except ValueError:
        print("Invalid input. Please enter a valid decimal integer.")

if not marked_states:
    print("No marked states provided. Exiting.")
    exit()

omega = np.array(marked_states)
M = len(omega)

dev = qml.device("default.qubit", wires=NUM_QUBITS)

def oracle(wires, omega):
    qml.FlipSign(omega, wires=wires)

def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

@qml.qnode(dev)
def circuit():
    iterations = int(np.round(np.sqrt(N / M) * np.pi / 4))

    equal_superposition(wires)

    for _ in range(iterations):
        for omg in omega:
            oracle(wires, omg)
        qml.templates.GroverOperator(wires)

    return qml.probs(wires)

probs = circuit()

'''Create a list of tuples (probability, binary_string)'''
state_probs = []
for index, prob in enumerate(probs):
    binary_string = format(index, f"0{NUM_QUBITS}b")
    if prob > 1e-6:  # Consider only probabilities greater than a small threshold
        state_probs.append((prob, binary_string))

# Sort in descending order
state_probs.sort(reverse=True, key=lambda x: x[0])

print("\nMeasured probabilities for each state (sorted in descending order):")
for prob, binary_string in state_probs:
    print(f"State {binary_string}: Probability {prob:.6f}")

In [ ]:
#Ishanya- 3 SAT with 4 variabkles
import time
from itertools import product

def _not(y):
    return y ^ 1

def sat_formula_large(x1, x2, x3, x4):
    return (
        (x2 | x3 | x4)
        & (_not(x1) | x2 | x3)
        & (_not(x1) | x2 | _not(x3))
        & (_not(x1) | _not(x2) | x3)
        & (x1 | _not(x2) | _not(x3))
        & (x1 | _not(x2) | x3)
        & (_not(x1) | _not(x2) | _not(x4))
        & (_not(x1) | _not(x2) | x4)
        & (_not(x2) | _not(x3) | _not(x4))
        & (x2 | _not(x3) | x4)
        & (x1 | _not(x3) | x4)
        & (x1 | _not(x2) | _not(x4))
        & (_not(x1) | _not(x2) | _not(x3))
    )

NUM_VARIABLES_LARGE = 4

def print_truth_table(num_vars, formula):
    """Print truth table and find solutions for the SAT formula"""
    print("\n=== Truth Table ===")
    print("x1 x2 x3 x4 | Result")
    print("-" * 16)

    solutions = []
    start_time = time.time()

    """ Generate all possible combinations"""
    for values in product([0, 1], repeat=num_vars):
        result = formula(*values)

    """ truth table """
        print(f" {values[0]}  {values[1]}  {values[2]}  {values[3]} |   {int(result)}")

        """ Store solution if formula is satisfied"""
        if result:
            solutions.append(values)

    end_time = time.time()

    print("\n=== Summary ===")
    print(f"Total combinations checked: {2**num_vars}")
    print(f"Solutions found: {len(solutions)}")
    print(f"Execution time: {(end_time - start_time):.6f} seconds")

    if solutions:
        print("\n=== Solutions ===")
        for i, sol in enumerate(solutions, 1):
            print(f"\nSolution {i}:")
            print(f"x1 = {sol[0]}")
            print(f"x2 = {sol[1]}")
            print(f"x3 = {sol[2]}")
            print(f"x4 = {sol[3]}")

        # solutions in binary format
        print("\nSolutions in binary format:")
        for sol in solutions:
            binary = ''.join(map(str, sol))
            print(binary)
    else:
        print("\nNo solutions found - formula is UNSATISFIABLE")

    ''' Analyze clause satisfaction'''
    print("\n=== Clause Analysis ===")
    if solutions:
        example_sol = solutions[0]
        x1, x2, x3, x4 = example_sol
        clauses = [
            (x2 | x3 | x4, "C1: (x2 ∨ x3 ∨ x4)"),
            (_not(x1) | x2 | x3, "C2: (¬x1 ∨ x2 ∨ x3)"),
            (_not(x1) | x2 | _not(x3), "C3: (¬x1 ∨ x2 ∨ ¬x3)"),
            (_not(x1) | _not(x2) | x3, "C4: (¬x1 ∨ ¬x2 ∨ x3)"),
            (x1 | _not(x2) | _not(x3), "C5: (x1 ∨ ¬x2 ∨ ¬x3)"),
            (x1 | _not(x2) | x3, "C6: (x1 ∨ ¬x2 ∨ x3)"),
            (_not(x1) | _not(x2) | _not(x4), "C7: (¬x1 ∨ ¬x2 ∨ ¬x4)"),
            (_not(x1) | _not(x2) | x4, "C8: (¬x1 ∨ ¬x2 ∨ x4)"),
            (_not(x2) | _not(x3) | _not(x4), "C9: (¬x2 ∨ ¬x3 ∨ ¬x4)"),
            (x2 | _not(x3) | x4, "C10: (x2 ∨ ¬x3 ∨ x4)"),
            (x1 | _not(x3) | x4, "C11: (x1 ∨ ¬x3 ∨ x4)"),
            (x1 | _not(x2) | _not(x4), "C12: (x1 ∨ ¬x2 ∨ ¬x4)"),
            (_not(x1) | _not(x2) | _not(x3), "C13: (¬x1 ∨ ¬x2 ∨ ¬x3)")
        ]

        print("Clause satisfaction for first solution:", ' '.join(map(str, example_sol)))
        for result, clause_text in clauses:
            print(f"{clause_text}: {'✓' if result else '✗'}")

if __name__ == "__main__":
    print("Solving large SAT formula...")
    print_truth_table(NUM_VARIABLES_LARGE, sat_formula_large)

In [ ]:
from collections import Counter

def _not(x):
    return not x

def sat_formula_large(x1, x2, x3, x4):
    return ((x2 or x3 or x4) and
            (not x1 or x2 or x3) and
            (not x1 or x2 or not x3) and
            (not x1 or not x2 or x3) and
            (x1 or not x2 or not x3) and
            (x1 or not x2 or x3) and
            (not x1 or not x2 or not x4) and
            (not x1 or not x2 or x4) and
            (not x2 or not x3 or not x4) and
            (x2 or not x3 or x4) and
            (x1 or not x3 or x4) and
            (x1 or not x2 or not x4) and
            (not x1 or not x2 or not x3))

class OraculeSAT:
    def __init__(self):
        self.aux_qubit_index = 4  # auxiliary qubits from index 4

    def get_auxiliary_qubit(self):
        qubit = self.aux_qubit_index
        self.aux_qubit_index += 1
        return qubit

    def or_gate(self, wire1, wire2, target):
        """Implements OR gate using Toffoli gates"""
        qml.PauliX(wires=wire1)
        qml.PauliX(wires=wire2)
        qml.Toffoli(wires=[wire1, wire2, target])
        qml.PauliX(wires=target)
        qml.PauliX(wires=wire1)
        qml.PauliX(wires=wire2)

    def or3_gate(self, wire1, wire2, wire3, target):
        """Implements 3-input OR gate"""
        aux = self.get_auxiliary_qubit()
        self.or_gate(wire1, wire2, aux)
        self.or_gate(aux, wire3, target)
        # Uncompute auxiliary qubit
        self.or_gate(wire1, wire2, aux)

    def clause_marker(self, x1, x2, x3, target, negations):
        """Marks a clause (x1 ∨ x2 ∨ x3) where each variable might be negated"""
        #  NOT gates for negated variables
        for wire, neg in zip([x1, x2, x3], negations):
            if neg:
                qml.PauliX(wires=wire)

        # OR operation
        self.or3_gate(x1, x2, x3, target)

        # Undo NOT gates
        for wire, neg in zip([x1, x2, x3], negations):
            if neg:
                qml.PauliX(wires=wire)

def oracle():
    """Complete oracle function for the SAT problem"""
    def inner_oracle(wires):
        oracle_helper = OraculeSAT()

        # Allocate result qubits for each clause
        clause_results = [oracle_helper.get_auxiliary_qubit() for _ in range(13)]

        # Implement each clause
        # (x2 ∨ x3 ∨ x4)
        oracle_helper.clause_marker(1, 2, 3, clause_results[0], [False, False, False])

        # (¬x1 ∨ x2 ∨ x3)
        oracle_helper.clause_marker(0, 1, 2, clause_results[1], [True, False, False])

        # (¬x1 ∨ x2 ∨ ¬x3)
        oracle_helper.clause_marker(0, 1, 2, clause_results[2], [True, False, True])

        # (¬x1 ∨ ¬x2 ∨ x3)
        oracle_helper.clause_marker(0, 1, 2, clause_results[3], [True, True, False])

        # (x1 ∨ ¬x2 ∨ ¬x3)
        oracle_helper.clause_marker(0, 1, 2, clause_results[4], [False, True, True])

        # (x1 ∨ ¬x2 ∨ x3)
        oracle_helper.clause_marker(0, 1, 2, clause_results[5], [False, True, False])

        # (¬x1 ∨ ¬x2 ∨ ¬x4)
        oracle_helper.clause_marker(0, 1, 3, clause_results[6], [True, True, True])

        # (¬x1 ∨ ¬x2 ∨ x4)
        oracle_helper.clause_marker(0, 1, 3, clause_results[7], [True, True, False])

        # (¬x2 ∨ ¬x3 ∨ ¬x4)
        oracle_helper.clause_marker(1, 2, 3, clause_results[8], [True, True, True])

        # (x2 ∨ ¬x3 ∨ x4)
        oracle_helper.clause_marker(1, 2, 3, clause_results[9], [False, True, False])

        # (x1 ∨ ¬x3 ∨ x4)
        oracle_helper.clause_marker(0, 2, 3, clause_results[10], [False, True, False])

        # (x1 ∨ ¬x2 ∨ ¬x4)
        oracle_helper.clause_marker(0, 1, 3, clause_results[11], [False, True, True])

        # (¬x1 ∨ ¬x2 ∨ ¬x3)
        oracle_helper.clause_marker(0, 1, 2, clause_results[12], [True, True, True])

        # Final AND operation using multi-controlled phase flip
        qml.MultiControlledPhaseFlip(wires=clause_results[0], ctrl_wires=clause_results[1:])

        """ Uncompute all clauses to clean auxiliary qubits
         (Same operations in reverse order)"""
        for i in range(12, -1, -1):
            if i == 0:
                oracle_helper.clause_marker(1, 2, 3, clause_results[i], [False, False, False])
            elif i == 1:
                oracle_helper.clause_marker(0, 1, 2, clause_results[i], [True, False, False])
            elif i == 2:
                oracle_helper.clause_marker(0, 1, 2, clause_results[i], [True, False, True])
            elif i == 3:
                oracle_helper.clause_marker(0, 1, 2, clause_results[i], [True, True, False])
            elif i == 4:
                oracle_helper.clause_marker(0, 1, 2, clause_results[i], [False, True, True])
            elif i == 5:
                oracle_helper.clause_marker(0, 1, 2, clause_results[i], [False, True, False])
            elif i == 6:
                oracle_helper.clause_marker(0, 1, 3, clause_results[i], [True, True, True])
            elif i == 7:
                oracle_helper.clause_marker(0, 1, 3, clause_results[i], [True, True, False])
            elif i == 8:
                oracle_helper.clause_marker(1, 2, 3, clause_results[i], [True, True, True])
            elif i == 9:
                oracle_helper.clause_marker(1, 2, 3, clause_results[i], [False, True, False])
            elif i == 10:
                oracle_helper.clause_marker(0, 2, 3, clause_results[i], [False, True, False])
            elif i == 11:
                oracle_helper.clause_marker(0, 1, 3, clause_results[i], [False, True, True])
            elif i == 12:
                oracle_helper.clause_marker(0, 1, 2, clause_results[i], [True, True, True])

    return inner_oracle

def diffusion():
    """Standard diffusion operator for Grover's algorithm"""
    n_qubits = 4  # number of qubits for input variables

    #H gates to all qubits
    for wire in range(n_qubits):
        qml.Hadamard(wires=wire)

    # X gates to all qubits
    for wire in range(n_qubits):
        qml.PauliX(wires=wire)

    # multi-controlled phase flip
    qml.MultiControlledPhaseFlip(wires=[0], ctrl_wires=[1, 2, 3])

    #  X gates to all qubits
    for wire in range(n_qubits):
        qml.PauliX(wires=wire)

    # H gates to all qubits
    for wire in range(n_qubits):
        qml.Hadamard(wires=wire)

""" Device-4 for variables + 16 auxiliary qubits"""
n_qubits = 20
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def grover_circuit(n_iterations):
    """Main Grover's algorithm circuit"""
    # Initialize
    for wire in range(4):  # Only initialize input qubits
        qml.Hadamard(wires=wire)

    for _ in range(n_iterations):
        oracle()(range(n_qubits))
        diffusion()

    # Measure
    return [qml.sample(wires=i) for i in range(4)]

def solve_sat():
    """Main function to solve the SAT problem"""
    n_iterations = int(np.pi/4 * np.sqrt(2**4))  # pi/4 * sqrt(N) for N=2^4

    n_shots = 100
    results = []
    for _ in range(n_shots):
        result = grover_circuit(n_iterations)
        results.append(tuple(result))

    # most common result!
    most_common = Counter(results).most_common(1)[0][0]

    # Convert to boolean
    solution = [bool(x) for x in most_common]

    # Verify
    if sat_formula_large(*solution):
        return solution
    return None

def main():
    """Main execution function"""
    print("Solving SAT formula using Grover's algorithm...")
    solution = solve_sat()

    if solution:
        print(f"\nSolution found:")
        print(f"x1 = {solution[0]}")
        print(f"x2 = {solution[1]}")
        print(f"x3 = {solution[2]}")
        print(f"x4 = {solution[3]}")
        print(f"\nVerification: {sat_formula_large(*solution)}")
    else:
        print("\nNo solution found")

if __name__ == "__main__":
    main()

In [ ]:
def _not(x):
    return not x

def sat_formula_large(x1, x2, x3, x4):
    return ((x2 or x3 or x4) and
            (not x1 or x2 or x3) and
            (not x1 or x2 or not x3) and
            (not x1 or not x2 or x3) and
            (x1 or not x2 or not x3) and
            (x1 or not x2 or x3) and
            (not x1 or not x2 or not x4) and
            (not x1 or not x2 or x4) and
            (not x2 or not x3 or not x4) and
            (x2 or not x3 or x4) and
            (x1 or not x3 or x4) and
            (x1 or not x2 or not x4) and
            (not x1 or not x2 or not x3))

def oracle():
    """Oracle function for the SAT problem"""
    def inner_oracle(wires):
        # Helper function to implement AND using Toffoli gates
        def and_gate(wire1, wire2, target):
            qml.Toffoli(wires=[wire1, wire2, target])

        # Helper function to implement OR using Toffoli and NOT gates
        def or_gate(wire1, wire2, target):
            qml.PauliX(wires=wire1)
            qml.PauliX(wires=wire2)
            qml.Toffoli(wires=[wire1, wire2, target])
            qml.PauliX(wires=target)
            qml.PauliX(wires=wire1)
            qml.PauliX(wires=wire2)

        """Implement the SAT formula using quantum gates
        We'll need auxiliary qubits for intermediate computations
        Main variables are in wires 0-3, auxiliary qubits start from wire 4 """

        # First clause: (x2 or x3 or x4)
        or_gate(1, 2, 4)
        or_gate(4, 3, 5)


        # Finally, flip the phase for satisfying assignments
        qml.PauliX(wires=5)
        qml.MultiControlledPhaseFlip(wires=[5], ctrl_wires=[0, 1, 2, 3])
        qml.PauliX(wires=5)

    return inner_oracle

def diffusion():
    """Diffusion operator for Grover's algorithm"""
    n_qubits = 4  # number of qubits for input variables

    #  H gates to all qubits
    for wire in range(n_qubits):
        qml.Hadamard(wires=wire)

    #  X gates to all qubits
    for wire in range(n_qubits):
        qml.PauliX(wires=wire)

    #  multi-controlled phase flip
    qml.MultiControlledPhaseFlip(wires=[0], ctrl_wires=[1, 2, 3])

    #  X gates to all qubits
    for wire in range(n_qubits):
        qml.PauliX(wires=wire)

    #  H gates to all qubits
    for wire in range(n_qubits):
        qml.Hadamard(wires=wire)

# device-4 for variables + 6 auxiliary qubits
n_qubits = 10  #
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def grover_circuit(n_iterations):
    """Main Grover's algorithm circuit"""
    # Initialize
    for wire in range(4):
        qml.Hadamard(wires=wire)

    for _ in range(n_iterations):
        oracle()(range(n_qubits))
        diffusion()

    # Measure
    return [qml.sample(wires=i) for i in range(4)]


def solve_sat():
    n_iterations = int(np.pi/4 * np.sqrt(2**4))  # pi/4 * sqrt(N) for N=2^4
    n_shots = 100
    results = []
    for _ in range(n_shots):
        result = grover_circuit(n_iterations)
        results.append(tuple(result))

    # most common result
    from collections import Counter
    most_common = Counter(results).most_common(1)[0][0]

    # Convert to boolean
    solution = [bool(x) for x in most_common]

    # Verify
    if sat_formula_large(*solution):
        return solution
    return None

solution = solve_sat()
print(f"Solution found: {solution}")
print(f"Verification: {sat_formula_large(*solution) if solution else 'No solution found'}")

#Rohan Mehra

In [ ]:
#Rohan Mehra

import pennylane as qml
from pennylane import numpy as np

# Parameters
N = 4 # Total number of database entries
n_qubits = int(np.log2(N))  # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define the target string
target_string = input(f"Enter the binary string to find ({n_qubits} bits): ")
assert len(target_string) == n_qubits, f"Target string must be {n_qubits} bits long."

# Convert the target string to the corresponding integer index
target_index = int(target_string, 2)

# Oracle Function
def oracle():
    """Marks the target state by flipping its sign."""
    binary = format(target_index, f"0{n_qubits}b")
    wires_to_flip = [i for i in range(n_qubits) if binary[i] == "0"]
    for wire in wires_to_flip:
        qml.PauliX(wire)
    qml.MultiControlledX(
        wires=list(range(n_qubits - 1)) + [n_qubits - 1],
        control_values=[1] * (n_qubits - 1),
    )
    for wire in wires_to_flip:
        qml.PauliX(wire)

# Diffusion Operator
def diffusion_operator():
    """Amplifies the probability of the marked state."""
    for i in range(n_qubits):
        qml.Hadamard(i)
        qml.PauliX(i)
    qml.MultiControlledX(
        wires=list(range(n_qubits - 1)) + [n_qubits - 1],
        control_values=[1] * (n_qubits - 1),
    )
    for i in range(n_qubits):
        qml.PauliX(i)
        qml.Hadamard(i)

# Grover's Algorithm
@qml.qnode(dev)
def grover_circuit(iterations):
    """Defines the Grover's search circuit."""
    # Step 1: Initialize in uniform superposition
    for i in range(n_qubits):
        qml.Hadamard(i)

    # Step 2: Apply Grover iterations
    for _ in range(iterations):
        oracle()
        diffusion_operator()

    return qml.probs(wires=range(n_qubits))

# Calculate the optimal number of iterations
iterations = int(np.floor(np.pi / 4 * np.sqrt(N)))
print(f"Optimal number of iterations: {iterations}")

# Display the circuit
@qml.qnode(dev)
def circuit_preview():
    """Displays the Grover's algorithm circuit structure."""
    for i in range(n_qubits):
        qml.Hadamard(i)
    for _ in range(iterations):
        oracle()
        diffusion_operator()
    return qml.probs(wires=range(n_qubits))

# Print the circuit
print("\n--- Grover's Algorithm Circuit ---")
print(qml.draw(circuit_preview)())

# Run the Grover's algorithm
print("\n--- Running Grover's Algorithm ---")
probs = grover_circuit(iterations)

# Display the results
print("\n--- Results ---")
for i, prob in enumerate(probs):
    binary = format(i, f"0{n_qubits}b")
    print(f"State {binary}: Probability {prob:.4f}")

In [ ]:
# Rohan Mehra

import pennylane as qml
from pennylane import numpy as np

# Ask for user input
N = int(input("Enter the number of database entries (N must be a power of 2): "))
assert (N & (N - 1)) == 0, "N must be a power of 2."  # Ensure N is a power of 2
n_qubits = int(np.log2(N))  # Number of qubits needed

marked_strings = input(
    f"Enter the binary strings to mark (separated by spaces, {n_qubits} bits each): "
).split()
marked_strings = [s for s in marked_strings if len(s) == n_qubits]

if not marked_strings:
    raise ValueError("You must provide at least one valid binary string to mark.")

# Convert marked strings to integers
marked_indices = [int(s, 2) for s in marked_strings]

# Define quantum device
dev = qml.device("default.qubit", wires=n_qubits)

# Oracle function
def oracle():
    for idx in marked_indices:
        binary = format(idx, f"0{n_qubits}b")
        wires_to_flip = [i for i in range(n_qubits) if binary[i] == "0"]
        for wire in wires_to_flip:
            qml.PauliX(wire)
        qml.MultiControlledX(wires=list(range(n_qubits)))
        for wire in wires_to_flip:
            qml.PauliX(wire)

# Diffusion operator
def diffusion_operator():
    for i in range(n_qubits):
        qml.Hadamard(i)
    for i in range(n_qubits):
        qml.PauliX(i)
    qml.MultiControlledX(wires=list(range(n_qubits)))
    for i in range(n_qubits):
        qml.PauliX(i)
    for i in range(n_qubits):
        qml.Hadamard(i)

# Grover's Algorithm
@qml.qnode(dev)
def grover_circuit(iterations):
    # Step 1: Initialize in uniform superposition
    for i in range(n_qubits):
        qml.Hadamard(i)

    # Step 2: Apply Grover iterations
    for _ in range(iterations):
        oracle()
        diffusion_operator()

    return qml.probs(wires=range(n_qubits))

# Calculate the number of iterations
M = len(marked_indices)
iterations = int(np.floor(np.pi / 4 * np.sqrt(N / M)))

# Run the circuit
print(f"Running Grover's algorithm with {iterations} iteration(s).")
probs = grover_circuit(iterations)

# Display results
for i, prob in enumerate(probs):
    binary = format(i, f"0{n_qubits}b")
    print(f"State {binary}: Probability {prob:.4f}")

In [ ]:
# Rohan Mehra

import pennylane as qml
from pennylane import numpy as np

NUM_QUBITS = 10

omega = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
])

M = len(omega)
N = 2**NUM_QUBITS
wires = list(range(NUM_QUBITS))

dev = qml.device("default.qubit", wires=NUM_QUBITS)

def oracle(wires, omega):
    qml.FlipSign(omega, wires=wires)

def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)

@qml.qnode(dev)
def circuit():
    iterations = int(np.round(np.sqrt(N / M) * np.pi / 4))

    # Initial state preparation
    equal_superposition(wires)

    # Grover's iterator
    for _ in range(iterations):
        for omg in omega:
            oracle(wires, omg)
        qml.templates.GroverOperator(wires)

    return qml.probs(wires=wires)


probs = circuit()

# Print each probability with its corresponding binary string
for index, prob in enumerate(probs):
    binary_string = format(index, f"0{NUM_QUBITS}b")
    print(f"State {binary_string}: Probability {prob:.6f}")